In [1]:
from nsepython import *   
import datetime

# key = "NIFTY"
# key = "BANKNIFTY"
# key = "SME"
# key = "FO"
# key = "OTHERS"
# key = "ALL"
import pandas as pd

def nse_preopen(key="NIFTY"):
    returnTo = []
    payload = nsefetch("https://www.nseindia.com/api/market-data-pre-open?key="+key+"")   
    dataList = payload['data']
    for d in dataList:
        obj = {}
        obj['lastUpdateTime'] = d['detail']['preOpenMarket']['lastUpdateTime']
        obj['symbol'] = d['metadata']['symbol']
        obj['lastPrice'] = d['metadata']['lastPrice']
        obj['change'] = d['metadata']['change']
        obj['pChange'] = d['metadata']['pChange']
        obj['previousClose'] = d['metadata']['previousClose']
        # obj['yearHigh'] = d['metadata']['yearHigh']
        # obj['yearLow'] = d['metadata']['yearLow']
        returnTo.append(obj)
    return returnTo

def get_option_chain(symbol):
    try:
        symbol = symbol.upper()
        option_chain_json = nse_optionchain_scrapper(symbol)
        data = option_chain_json['filtered']['data']
        return data
    except:
        pass

def get_atm_strike(option_chain_data):
    try:
        ltp = option_chain_data[0]['PE']['underlyingValue']
        strike_price_list = [x['strikePrice'] for x in option_chain_data]
        atm_strike = sorted([[round(abs(ltp-i),2),i] for i in strike_price_list])[0][1]
        return atm_strike
    except:
        return 0

def get_pe_ce_price(atm_strike, option_chain_data):
    for dictt in option_chain_data:
        if dictt['strikePrice'] == atm_strike:
            pe_price = dictt['PE']['askPrice']
            ce_price = dictt['CE']['askPrice']
            return pe_price,ce_price
        
def getTokenDf():
    token_df = pd.read_json("./OpenAPIScripMaster.json")
    token_df['expiry'] = pd.to_datetime(token_df['expiry']).apply(lambda x: x.date())
    token_df = token_df.astype({'strike': float})
    return token_df

def getTokenInfo (symbol, exch_seg ='NSE',instrumenttype='OPTIDX',strike_price = '',pe_ce = 'CE',expiry_day = None):
    df = getTokenDf()
    strike_price = strike_price*100
    if exch_seg == 'NSE':
        eq_df = df[(df['exch_seg'] == 'NSE') ]
        return eq_df[eq_df['name'] == symbol]
    elif exch_seg == 'NFO' and ((instrumenttype == 'FUTSTK') or (instrumenttype == 'FUTIDX')):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol)].sort_values(by=['expiry'])
    elif exch_seg == 'NFO' and (instrumenttype == 'OPTSTK' or instrumenttype == 'OPTIDX'):
        return df[(df['exch_seg'] == 'NFO') & (df['expiry']==expiry_day) &  (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol) & (df['strike'] == strike_price) & (df['symbol'].str.endswith(pe_ce))].sort_values(by=['expiry'])

        
preeopen_df = pd.DataFrame(nse_preopen("NIFTY"))
f_preeopen_df = preeopen_df.head(10)


pre_dict = f_preeopen_df.to_dict('index')

pre_dict

{0: {'lastUpdateTime': '15-Nov-2022 09:07:17',
  'symbol': 'COALINDIA',
  'lastPrice': 234.4,
  'change': 2.200000000000017,
  'pChange': 0.9474590869939782,
  'previousClose': 247.2},
 1: {'lastUpdateTime': '15-Nov-2022 09:07:16',
  'symbol': 'ICICIBANK',
  'lastPrice': 900.55,
  'change': 5.599999999999909,
  'pChange': 0.625733281188883,
  'previousClose': 894.95},
 2: {'lastUpdateTime': '15-Nov-2022 09:07:22',
  'symbol': 'ONGC',
  'lastPrice': 140.0,
  'change': 0.75,
  'pChange': 0.5385996409335727,
  'previousClose': 139.25},
 3: {'lastUpdateTime': '15-Nov-2022 09:07:17',
  'symbol': 'ITC',
  'lastPrice': 348.95,
  'change': 1.75,
  'pChange': 0.5040322580645161,
  'previousClose': 347.2},
 4: {'lastUpdateTime': '15-Nov-2022 09:07:17',
  'symbol': 'HDFCBANK',
  'lastPrice': 1623.15,
  'change': 8.100000000000136,
  'pChange': 0.5015324602953554,
  'previousClose': 1615.05},
 5: {'lastUpdateTime': '15-Nov-2022 09:07:17',
  'symbol': 'SUNPHARMA',
  'lastPrice': 1022.0,
  'change':

In [7]:
from datetime import datetime,date

f_preopen = []
expiry_day = date(2022,11,24)

for i in pre_dict.keys():
    actual = pre_dict.get(i)
    try: 
        option_chain_data = get_option_chain(actual['symbol'])
        actual['atm_strike'] = get_atm_strike(option_chain_data)
        if actual['atm_strike'] != 0:
            
            ce_ltp, pe_ltp = get_pe_ce_price(actual['atm_strike'], option_chain_data)
            actual['CE_LTP'] = ce_ltp
            actual['SL'] = ce_ltp * 1.10
            actual['PE_LTP'] = pe_ltp
            tokenInfo = getTokenInfo(actual['symbol'], 'NFO', 'OPTSTK', actual['atm_strike'], 'CE', expiry_day).iloc[0]
            actual['lotsize'] = int(tokenInfo['lotsize'])
            actual['token'] = int(tokenInfo['token'])
            actual['full_symbol'] = tokenInfo['symbol']
            actual['funds_req_ce_buy'] = ce_ltp * actual['lotsize']
        f_preopen.append(actual)
    except:
        pass

fdf =  pd.DataFrame.from_dict(f_preopen)
fdf

,lastUpdateTime,symbol,lastPrice,change,pChange,previousClose,atm_strike,CE_LTP,SL,PE_LTP,lotsize,token,full_symbol,funds_req_ce_buy
0,14-Nov-2022 09:08:00,M&M,1304.00,16.90,1.313029,1287.10,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14-Nov-2022 09:08:00,HINDALCO,435.00,5.15,1.198092,429.85,455.0,10.60,11.660,11.65,1075.0,80827.0,HINDALCO24NOV22455CE,11395.0
2,14-Nov-2022 09:08:01,POWERGRID,210.95,1.90,0.908873,214.05,212.5,4.05,4.455,3.95,2700.0,117257.0,POWERGRID24NOV22212.5CE,10935.0
3,14-Nov-2022 09:08:01,TATASTEEL,108.50,0.95,0.883310,107.55,109.0,2.30,2.530,2.60,4250.0,144794.0,TATASTEEL24NOV22109CE,9775.0
4,14-Nov-2022 09:08:01,WIPRO,402.65,2.95,0.738054,399.70,395.0,4.80,5.280,8.60,1000.0,127792.0,WIPRO24NOV22395CE,4800.0
5,14-Nov-2022 09:08:06,NTPC,172.35,0.90,0.524934,171.45,172.5,2.85,3.135,3.15,5700.0,108738.0,NTPC24NOV22172.5CE,16245.0
6,14-Nov-2022 09:08:01,COALINDIA,254.55,1.30,0.513327,253.25,247.5,5.00,5.500,6.45,4200.0,70065.0,COALINDIA24NOV22247.5CE,21000.0
7,14-Nov-2022 09:08:01,SUNPHARMA,1018.65,5.10,0.503182,1013.55,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,14-Nov-2022 09:08:00,HDFC,2665.00,13.30,0.501565,2651.70,2660.0,38.95,42.845,40.95,300.0,78926.0,HDFC24NOV222660CE,11685.0
9,14-Nov-2022 09:08:01,UPL,771.75,3.85,0.501367,767.90,770.0,9.20,10.120,16.00,1300.0,127352.0,UPL24NOV22770CE,11960.0


In [ ]:
def placeBuyOrder(script, token, price, sl):    
    orderparams = {
                "variety": "NORMAL",
                "tradingsymbol": script,
                "symboltoken": int(token),
                "transactiontype": "BUY",
                "exchange": "NFO",
                "ordertype": "LIMIT",
                "producttype": "INTRADAY",
                "duration": "DAY",
                "price": price,
                "squareoff": "0",
                "stoploss": str(sl),
                "quantity": 1
            }

In [9]:
preopen_loosers_df = preeopen_df.tail(10).sort_values(['pChange'], ascending=True)
preopen_loosers_df

,lastUpdateTime,symbol,lastPrice,change,pChange,previousClose
49,14-Nov-2022 09:08:00,DRREDDY,4400.00,-139.45,-3.071958,4539.45
48,14-Nov-2022 09:08:01,BHARTIARTL,818.00,-8.90,-1.076309,826.90
47,14-Nov-2022 09:08:00,ADANIENT,3999.00,-9.85,-0.245706,4008.85
46,14-Nov-2022 09:08:01,ICICIBANK,905.00,-1.55,-0.170978,906.55
45,14-Nov-2022 09:08:01,RELIANCE,2630.75,-1.05,-0.039897,2631.80
44,14-Nov-2022 09:08:00,AXISBANK,850.50,-0.20,-0.023510,850.70
43,14-Nov-2022 09:08:00,ITC,356.35,0.00,0.000000,356.35
42,14-Nov-2022 09:08:02,NESTLEIND,20323.00,0.50,0.002460,20322.50
41,14-Nov-2022 09:08:00,TATACONSUM,773.35,0.15,0.019400,773.20
40,14-Nov-2022 09:08:00,ASIANPAINT,3056.00,0.60,0.019637,3055.40
